In [1]:
import cv2
import numpy as np

# Load the Haar cascade file for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Create a variable to store face data
individuals = []

def capture_faces(name):
    cap = cv2.VideoCapture(0)
    face_data = []
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image")
            break
        
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_frame, 1.3, 5)
        
        if len(faces) > 0:
            # Sort faces by size (largest first)
            faces = sorted(faces, key=lambda x: x[2]*x[3], reverse=True)
            (x, y, w, h) = faces[0]
            
            # Extract the face region and resize it
            face = frame[y:y+h, x:x+w]
            face = cv2.resize(face, (100, 100))
            
            # Save every 3rd frame
            if frame_count % 3 == 0:
                face_data.append(face)
                print(f"Saved frame {frame_count // 3 + 1}")
            
            # Draw a rectangle around the face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        cv2.imshow("Face Capture", frame)
        
        frame_count += 1

        # Break the loop after capturing 20 frames
        if frame_count >= 18:  # 20 frames saved every 3rd frame
            break

        # Press 'q' to break the loop early
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # Convert face data to numpy array and save
    face_data = np.asarray(face_data)
    np.save(f'{name}.npy', face_data)
    print(f"Collected {len(face_data)} face images and saved as {name}.npy")

while True:
    name = input('Enter your name: ').capitalize()
    individuals.append(name)
    capture_faces(name)
    
    a = input('Do you want to add another student? (Yes/No): ').capitalize()
    if a == 'No':
        break

Enter your name: Dinesh 
Collected 0 face images and saved as Dinesh .npy
Do you want to add another student? (Yes/No): Yes
Enter your name: Raghu
Collected 0 face images and saved as Raghu.npy
Do you want to add another student? (Yes/No): Yes
Enter your name: Lavanya
Collected 0 face images and saved as Lavanya.npy
Do you want to add another student? (Yes/No): Yes
Enter your name: Vamshi
Collected 0 face images and saved as Vamshi.npy
Do you want to add another student? (Yes/No): No


In [2]:
import pandas as pd
import numpy as np
import keras
import seaborn as sns
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Input, Dense, Conv2D, MaxPooling2D,Flatten

In [3]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(individuals), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

C:\Users\deepa\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
model.save('face_recognition_model.h5')

In [5]:
import csv
from datetime import datetime

# Load the Haar cascade file for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize video capture
cap = cv2.VideoCapture(0)

# Open a CSV file to record attendance
with open('attendance.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Name", "Time"])

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image")
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_frame, 1.3, 5)

        for (x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]
            face = cv2.resize(face, (100, 100))
            face = face / 255.0
            face = np.expand_dims(face, axis=0)

            # Predict the face
            prediction = model.predict(face)
            label_index = np.argmax(prediction)
            person_name = label_map[label_index]

            # Draw rectangle around the face and label it
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, person_name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

            # Record attendance
            writer.writerow([person_name, datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
            print(f"Recorded attendance for {person_name}")

        cv2.imshow("Attendance System", frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 439ms/step


NameError: name 'label_map' is not defined